# Difinity Code: Fabric Admin & Governance
**Presentation Demo**

This notebook demonstrates how to use `semantic-link-labs` to automate administrative tasks, audit security, and manage tenant settings in Microsoft Fabric.

### Prerequisites
- **Library:** `semantic-link-labs` must be installed.
- **Permissions:** Fabric Administrator role is required for most operations.
- **Service Principal:** Required for the Graph API section (Client ID/Secret).

> **Documentation:** [Semantic Link Labs Installation](https://semantic-link-labs.readthedocs.io/en/stable/)

In [ ]:
# Install the required library
%pip install semantic-link-labs

In [ ]:
import pandas as pd
import sempy_labs as labs
from sempy_labs import admin
from datetime import datetime, timedelta, timezone
import json
from importlib.metadata import version

# Configure pandas to display all columns for better visibility
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Retrieve version using importlib instead of direct attribute access
try:
    lib_version = version("semantic-link-labs")
    print("Setup complete. Library version:", lib_version)
except Exception as e:
    print("Setup complete. (Could not determine version number)")

## 1. Workspace Inventory & Scanner API

This section retrieves a list of all workspaces in the tenant. It then utilizes the **Scanner API** to fetch deep metadata, including dataset schemas and lineage information.

> **Note:** The `scan_workspaces` API has a limit of 100 workspaces per request. The code below includes logic to handle batching if necessary, though the default example scans a specific subset.

**Reference:** [admin.scan_workspaces](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.admin.html#sempy_labs.admin.scan_workspaces)

In [ ]:
# 1. List all workspaces in the tenant
df_workspaces = admin.list_workspaces()

# 2. Extract IDs (IDs are more reliable than names)
# For this demo, we will take the first 50 to ensure quick execution
workspace_ids = df_workspaces['Id'].tolist()[:50] 

print(f"Preparing to scan {len(workspace_ids)} workspaces...")

# 3. Run the Workspace Scan
# dataset_schema=True fetches table/column details
# lineage=True fetches upstream/downstream dependencies
scan_result = admin.scan_workspaces(
    workspace=workspace_ids,
    dataset_schema=True,      
    lineage=True              
)

print("Scan complete.")

# 4. Display the result (it returns a DataFrame or Dictionary depending on version)
if isinstance(scan_result, pd.DataFrame):
    display(scan_result.head())
else:
    print("Scan returned object type:", type(scan_result))

In [ ]:
#Show the json object with all the scan details for the tenant
scan_result

## 2. Workspace Security & Access

Understanding who has access to your data is critical. This function retrieves the **Role Assignments** (Admin, Member, Contributor, Viewer) for a specific workspace.

**Reference:** [labs.list_workspace_role_assignments](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.list_workspace_role_assignments)

In [ ]:
# Replace with a valid Workspace ID or Name from your tenant
target_workspace = "Sales" 

try:
    roles_df = labs.list_workspace_role_assignments(workspace=target_workspace)
    display(roles_df)
except Exception as e:
    print(f"Error accessing workspace '{target_workspace}': {e}")

## 3. Tenant Auditing: Activity Events

This section fetches the **Activity Events** (Audit Logs) for the tenant. This is useful for identifying who accessed what report, or what API operations were performed.

> **Note:** The API requires start and end times in UTC. This script defaults to "Yesterday (UTC)".

**Reference:** [admin.list_activity_events](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.admin.html#sempy_labs.admin.list_activity_events)

In [ ]:
# Calculate UTC time range for yesterday
utc_now = datetime.now(timezone.utc)
yesterday = (utc_now - timedelta(days=1)).date()

start_str = f"{yesterday.strftime('%Y-%m-%d')}T00:00:00"
end_str   = f"{yesterday.strftime('%Y-%m-%d')}T23:59:59"

print(f"Fetching events for: {start_str} to {end_str} (UTC)")

# Fetch events
events_df = admin.list_activity_events(start_time=start_str, end_time=end_str)

# Normalize JSON data if necessary
if not isinstance(events_df, pd.DataFrame):
    events_df = pd.json_normalize(events_df)

print(f"Fetched {len(events_df):,} events.")
display(events_df.head(5))

## 4. Cleanup: Identify Unused Artifacts

This function scans a workspace to identify artifacts (Reports, Datasets, Dashboards) that have not been accessed recently. This is essential for tenant hygiene and cost management.

**Reference:** [admin.list_unused_artifacts](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.admin.html#sempy_labs.admin.list_unused_artifacts)

In [ ]:
target_workspace_cleanup = "Sales" # Replace with your workspace name

try:
    # Returns artifacts not accessed in the last 30 days (default)
    unused_df = admin.list_unused_artifacts(workspace=target_workspace_cleanup)
    
    if not unused_df.empty:
        print("Found the following unused artifacts:")
        display(unused_df)
    else:
        print("No unused artifacts found.")
        
except Exception as e:
    print(f"Could not scan workspace: {e}")

## 5. Tenant Settings: Manage Preview Features

This script discovers all Tenant Settings that are currently in **Preview** and are **Enabled**. It demonstrates how to programmatically list settings.

> **Warning:** The code to *disable* settings is included but commented out for safety. Updating tenant settings affects all users immediately.

**Reference:** [admin.list_tenant_settings](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.admin.html#sempy_labs.admin.list_tenant_settings)

In [ ]:
import sempy.fabric as fabric

# 1. Get all tenant settings
settings_df = admin.list_tenant_settings()

# 2. Filter for 'Enabled' features that contain '(preview)' in the title
preview_enabled = settings_df[
    (settings_df['Title'].str.contains(r'\(preview\)', case=False, na=False)) & 
    (settings_df['Enabled'] == True)
]

print(f"Found {len(preview_enabled)} enabled preview features.")
display(preview_enabled[['Setting Name', 'Title', 'Enabled']])

# 3. (Optional) Bulk Disable Logic
# client = fabric.FabricRestClient()
# for index, row in preview_enabled.iterrows():
#     setting_name = row['Setting Name']
#     try:
#         print(f"Disabling: {setting_name}")
#         # client.post(f"/v1/admin/tenantsettings/{setting_name}/update", 
#         #             json={"enabled": False, "enabledSecurityGroups": []})
#     except Exception as e:
#         print(f"Failed to update {setting_name}: {e}")

## 6. Optimization: Best Practice Analyzer (BPA)

The BPA scans Semantic Models within a workspace against a set of standard rules (performance, formatting, DAX expressions) to ensure model quality.

An lakehouse is required and acts as the destination for the BPA results

**Reference:** [labs.run_model_bpa_bulk](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.run_model_bpa_bulk)

In [ ]:
target_workspace_bpa = "Sales"

print(f"Running BPA on: {target_workspace_bpa}")

try:
    bpa_results = labs.run_model_bpa_bulk(workspace=target_workspace_bpa)
    display(bpa_results)
except Exception as e:
    print(f"BPA Run failed: {e}")

In [ ]:
from deltalake import DeltaTable, write_deltalake
table_path = 'abfss://76199f5a-1ec6-4dd7-9751-0e79c93d5319@onelake.dfs.fabric.microsoft.com/ce5ef1f1-6e19-416e-9c74-d48fff22f1ee/Tables/modelbparesults' 
storage_options = {"bearer_token": notebookutils.credentials.getToken('storage'), "use_fabric_endpoint": "true"}
dt = DeltaTable(table_path, storage_options=storage_options)
limited_data = dt.to_pyarrow_dataset().head(1000).to_pandas()
display(limited_data)

# Write data frame to Lakehouse
# write_deltalake(table_path, limited_data, mode='overwrite')

# If the table is too large and might cause an Out of Memory (OOM) error,
# you can try using the code below. However, please note that delta_scan with default lakehouse is currently in preview.
# import duckdb
# display(duckdb.sql("select * from delta_scan('/lakehouse/default/Tables/dbo/bigdeltatable') limit 1000 ").df())

## 7. Strategy Check: Apps vs. Workspaces

This script analyzes the Activity Events (fetched in Block 4) to calculate the **"App Adoption Rate"**. It checks how many report views came through an App versus directly through the Workspace.

**Reference:** [admin.list_apps](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.admin.html#sempy_labs.admin.list_apps)

In [ ]:
# Ensure you have run Block 4 (Activity Events) first to get 'events_df'

print("Analyzing Report Consumption Methods...")

# 1. Filter for "ViewReport" operations only
view_events = events_df[events_df['Operation'] == 'ViewReport'].copy()

# 2. Determine if the view was via an App
# Note: The field name in audit logs can vary, but 'AppId' is usually present if viewed via App
if 'AppId' in view_events.columns:
    view_events['IsAppView'] = view_events['AppId'].notna() & (view_events['AppId'] != '')
else:
    # Fallback logic if AppId column is missing in the raw feed
    view_events['IsAppView'] = False 

# 3. Calculate Stats
total_views = len(view_events)
app_views = len(view_events[view_events['IsAppView'] == True])
workspace_views = len(view_events[view_events['IsAppView'] == False])

if total_views > 0:
    adoption_rate = (app_views / total_views) * 100
    print(f"Total Report Views: {total_views}")
    print(f"✅ Views via App:     {app_views} ({adoption_rate:.1f}%)")
    print(f"❌ Views via Workspace: {workspace_views} ({100 - adoption_rate:.1f}%)")
    
    if adoption_rate < 50:
        print("\n⚠️ Insight: The majority of users are accessing reports directly in Workspaces. Review 'Workspace Viewer' roles.")
else:
    print("No 'ViewReport' events found in the selected time range.")

# 4. (Optional) List all Apps in the Tenant for Inventory
# apps_df = admin.list_apps()
# display(apps_df.head())